In [1]:
import logging
logging.getLogger('tensorflow').disabled = False

from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
import sys
import numpy as np
import nltk
from importlib import reload
import os

sys.path.append('../src')
from study_design_type.study_type_labels import StudyTypeLabels
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
from utilities import excel_reader, excel_writer
from sklearn.utils import shuffle
#import eli5

C:\Users\Maryia_Ivanina\AppData\Local\Continuum\anaconda3\envs\cornell\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Maryia_Ivanina\AppData\Local\Continuum\anaconda3\envs\cornell\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Maryia_Ivanina\AppData\Local\Continuum\anaconda3\envs\cornell\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.d

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [3]:
!pip install bert-tensorflow

In [2]:
from sklearn.utils import shuffle

def load_train_study_type(label, proportions_to_use=None):
    all_df = pd.DataFrame()
    test = None
    replicated_df = pd.DataFrame()
    for file in os.listdir("../tmp/study_type_data"):
        if file == "final_study_type_test_data.xlsx":
            continue
        replicate = 1
        if file == "labelled_multi.xlsx":
            replicate = 4
        print(file, replicate)
        if file == "valid_data.xlsx":
            test = excel_reader.ExcelReader().read_df_from_excel(os.path.join("../tmp/study_type_data", file))
            test["label"] = 0
            for i in range(len(test)):
                test["label"].values[i] = 1 if test[label].values[i] >= 0.5 else 0
            test = test[["title", "abstract", "label"]]
        if replicate > 1:
            for i in range(replicate):
                train_1 = excel_reader.ExcelReader().read_df_from_excel(os.path.join("../tmp/study_type_data", file))
                train_1["label"] = 0
                for i in range(len(train_1)):
                    train_1["label"].values[i] = 1 if train_1[label].values[i] >= 0.5 else 0
                replicated_df = pd.concat([replicated_df, train_1[["title", "abstract", "label"]]],axis=0)
        else:
            for i in range(replicate):
                train_1 = excel_reader.ExcelReader().read_df_from_excel(os.path.join("../tmp/study_type_data", file))
                train_1["label"] = 0
                for i in range(len(train_1)):
                    train_1["label"].values[i] = 1 if train_1[label].values[i] >= 0.5 else 0
                all_df = pd.concat([all_df, train_1[["title", "abstract", "label"]]],axis=0)
    df_merged = pd.concat([all_df, replicated_df],axis=0)
    count_ones = len(df_merged[df_merged["label"] == 1])
    count_zeros_to_take = len(all_df)
    if proportions_to_use is not None:
        count_zeros = count_ones * proportions_to_use
        count_zeros_to_take = count_zeros - len(replicated_df[replicated_df["label"] == 0])
        count_zeros_to_take = max(0, count_zeros_to_take)
    all_df = pd.concat([all_df[all_df["label"] == 1], shuffle(all_df[all_df["label"] == 0])[:count_zeros_to_take], replicated_df],axis=0)
    print("Train ", len(all_df))
    print(all_df["label"].value_counts())
    print("Test ", len(test))
    print(test["label"].value_counts())
    all_df = shuffle(all_df)
    return all_df, test

In [3]:
from sklearn.utils import shuffle

priority_labels = {"No category": 0, "Field study": 0, "Laboratory study": 1, 'Modeling study': 3, 'Observational study': 2, 'Review paper': 4}

study_labels = ["Observational study", "Modeling study", "Laboratory study", "Review paper", "Field study"]

def create_one_label_df(test):
    test["label"] = 5
    for i in range(len(test)):
        priority_labels_data = [
            (StudyTypeLabels.STUDY_TYPE_LABEL_TO_NUMBER[column], priority_labels[column]) for column in test.columns if column in priority_labels and test[column].values[i] >= 0.5]
        if len(priority_labels_data):
            test["label"].values[i] = sorted(priority_labels_data, key=lambda x: x[1])[0][0]
    test = test[["title", "abstract", "label"]]
    return test

def return_all_df_labels(test):
    test["label"] = ""
    for i in range(len(test)):
        priority_labels_data = [
            (StudyTypeLabels.STUDY_TYPE_LABEL_TO_NUMBER[column], priority_labels[column]) for column in test.columns if column in priority_labels and test[column].values[i] >= 0.5]
        if len(priority_labels_data):
            test["label"].values[i] = [r[0] for r in sorted(priority_labels_data, key=lambda x: x[1])]
        else:
            test["label"].values[i] = [5]
    test = test[["title", "abstract", "label"]]
    return test

def load_train_study_type_one_label():
    all_df = pd.DataFrame()
    test = None
    for file in os.listdir("../tmp/study_type_data"):
        replicate = 1
        if file == "labelled_multi.xlsx":
            replicate = 4
        print(file, replicate)
        if file == "valid_data.xlsx":
            test = excel_reader.ExcelReader().read_df_from_excel(os.path.join("../tmp/study_type_data", file))
            test = create_one_label_df(test)
        for i in range(replicate):
            train_1 = excel_reader.ExcelReader().read_df_from_excel(os.path.join("../tmp/study_type_data", file))
            train_1 = create_one_label_df(train_1)
            all_df = pd.concat([all_df, train_1[["title", "abstract", "label"]]],axis=0)
    print("Train ", len(all_df))
    print(all_df["label"].value_counts())
    print("Test ", len(test))
    print(test["label"].value_counts())
    all_df = shuffle(all_df)
    return all_df, test

def create_multi_label_target(df):
    df["label"] = ""
    for i in range(len(df)):
        target_labels = []
        for column in study_labels:
            target_labels.append(df[column].values[i])
        df["label"].values[i] = target_labels
    return df[["title", "abstract", "label"]]

def load_train_study_type_multi_label():
    all_df = pd.DataFrame()
    test = None
    for file in os.listdir("../tmp/study_type_data"):
        replicate = 1
        if file == "labelled_multi.xlsx":
            replicate = 4
        print(file, replicate)
        if file == "valid_data.xlsx":
            test = excel_reader.ExcelReader().read_df_from_excel(os.path.join("../tmp/study_type_data", file))
            test = create_multi_label_target(test)
        for i in range(replicate):
            train_1 = excel_reader.ExcelReader().read_df_from_excel(os.path.join("../tmp/study_type_data", file))
            train_1 = create_multi_label_target(train_1)
            all_df = pd.concat([all_df, train_1],axis=0)
    print("Train ", len(all_df))
    print("Test ", len(test))
    all_df = shuffle(all_df)
    return all_df, test

In [4]:
train, test = load_train_study_type_multi_label()
test_plus_stefan_all_labels = create_multi_label_target(excel_reader.ExcelReader().read_df_from_excel("../tmp/study_type_data/final_study_type_test_data.xlsx"))

final_study_type_test_data.xlsx 1
Read file ../tmp/study_type_data\final_study_type_test_data.xlsx: 0.05s
Processed file ../tmp/study_type_data\final_study_type_test_data.xlsx: 0.01s
labelled_multi.xlsx 4
Read file ../tmp/study_type_data\labelled_multi.xlsx: 0.12s
Processed file ../tmp/study_type_data\labelled_multi.xlsx: 0.04s
Read file ../tmp/study_type_data\labelled_multi.xlsx: 0.11s
Processed file ../tmp/study_type_data\labelled_multi.xlsx: 0.03s
Read file ../tmp/study_type_data\labelled_multi.xlsx: 0.11s
Processed file ../tmp/study_type_data\labelled_multi.xlsx: 0.04s
Read file ../tmp/study_type_data\labelled_multi.xlsx: 0.23s
Processed file ../tmp/study_type_data\labelled_multi.xlsx: 0.06s
train_keywords_plus_multi.xlsx 1
Read file ../tmp/study_type_data\train_keywords_plus_multi.xlsx: 2.10s
Processed file ../tmp/study_type_data\train_keywords_plus_multi.xlsx: 1.06s
valid_data.xlsx 1
Read file ../tmp/study_type_data\valid_data.xlsx: 0.10s
Processed file ../tmp/study_type_data\val

In [6]:
train, test = load_train_study_type_one_label()
test_plus_stefan_one_label = create_one_label_df(excel_reader.ExcelReader().read_df_from_excel("../tmp/study_type_data/final_study_type_test_data.xlsx"))
test_plus_stefan_all_labels = return_all_df_labels(excel_reader.ExcelReader().read_df_from_excel("../tmp/study_type_data/final_study_type_test_data.xlsx"))

final_study_type_test_data.xlsx 1
Read file ../tmp/study_type_data\final_study_type_test_data.xlsx: 1.08s
Processed file ../tmp/study_type_data\final_study_type_test_data.xlsx: 0.02s
labelled_multi.xlsx 4
Read file ../tmp/study_type_data\labelled_multi.xlsx: 0.24s
Processed file ../tmp/study_type_data\labelled_multi.xlsx: 0.08s
Read file ../tmp/study_type_data\labelled_multi.xlsx: 0.19s
Processed file ../tmp/study_type_data\labelled_multi.xlsx: 0.06s
Read file ../tmp/study_type_data\labelled_multi.xlsx: 0.18s
Processed file ../tmp/study_type_data\labelled_multi.xlsx: 0.07s
Read file ../tmp/study_type_data\labelled_multi.xlsx: 0.18s
Processed file ../tmp/study_type_data\labelled_multi.xlsx: 0.05s
train_keywords_plus_multi.xlsx 1
Read file ../tmp/study_type_data\train_keywords_plus_multi.xlsx: 2.64s
Processed file ../tmp/study_type_data\train_keywords_plus_multi.xlsx: 1.62s
valid_data.xlsx 1
Read file ../tmp/study_type_data\valid_data.xlsx: 0.17s
Processed file ../tmp/study_type_data\val

In [11]:
r = _study_type_labeler.predict_with_meta_model(test_plus_stefan_one_label, with_head_tail = True, recreate_model = True)

Used for model gpu 0

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Used for model gpu 0
Used for model gpu 0


In [13]:
true_results = []
for i in range(len(r[1])):
    if r[1][i] in test_plus_stefan_all_labels["label"].values[i]:
        true_results.append(r[1][i])
    else:
        true_results.append(test_plus_stefan_one_label["label"].values[i])
_study_type_labeler.print_summary(true_results, r[1])

[[75  3  1  2  6  4]
 [ 7 29  0  1  2  0]
 [ 2  0 22  0  1  3]
 [ 4  0  0 26  1  1]
 [ 5  0  0  1 24  0]
 [ 7  0  0  0  1 22]]
              precision    recall  f1-score   support

           0       0.75      0.82      0.79        91
           1       0.91      0.74      0.82        39
           2       0.96      0.79      0.86        28
           3       0.87      0.81      0.84        32
           4       0.69      0.80      0.74        30
           5       0.73      0.73      0.73        30

   micro avg       0.79      0.79      0.79       250
   macro avg       0.82      0.78      0.80       250
weighted avg       0.80      0.79      0.79       250

F1 score:  0.795915228306712


In [8]:
_study_type_labeler.print_summary(test_plus_stefan_one_label["label"], r[1])

[[73 14  1  4  6  4]
 [ 7 14  0  2  2  0]
 [ 4  3 21  0  1  3]
 [ 4  0  0 23  1  1]
 [ 5  1  1  1 24  0]
 [ 7  0  0  0  1 22]]
              precision    recall  f1-score   support

           0       0.73      0.72      0.72       102
           1       0.44      0.56      0.49        25
           2       0.91      0.66      0.76        32
           3       0.77      0.79      0.78        29
           4       0.69      0.75      0.72        32
           5       0.73      0.73      0.73        30

   micro avg       0.71      0.71      0.71       250
   macro avg       0.71      0.70      0.70       250
weighted avg       0.72      0.71      0.71       250

F1 score:  0.7011748286282953


In [5]:
from bert_models import base_bert_model_meta_model
base_bert_model_meta_model = reload(base_bert_model_meta_model)
from bert_models import base_bert_model_meta_model_with_window_all
base_bert_model_meta_model_with_window_all = reload(base_bert_model_meta_model_with_window_all)
from study_design_type import study_type_labeler
study_type_labeler = reload(study_type_labeler)
from study_design_type import study_type_labeler_window_agg
study_type_labeler_window_agg = reload(study_type_labeler_window_agg)

In [13]:
_study_type_labeler = study_type_labeler.StudyTypeLabeler(
      "study_type_multi",
      max_seq_length = 256, batch_size = 16,
      gpu_device_num_hub = 0, gpu_device_num = 0,
      model_folder = "../tmp/scibert_scivocab_uncased",
      label_list = list(range(len(study_labels))),
      meta_model_folder = "study_type_multi_meta",
      use_one_layer=True,
      keep_prob=0.8,
      epochs_num=5,
      multilabel=True
      )
#_study_type_labeler.train_model(train, test, use_tails=True)
_study_type_labeler.train_meta_model(train, test, use_tail=True)

Started tokenizer loading
Used gpu 0
Tokenizrer loaded
0
../tmp/scibert_scivocab_uncased
INFO:tensorflow:Using config: {'_model_dir': 'study_type_multi', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001F31DA00C50>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': 'study_type_multi', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001F31DA00C50>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Config is done
INFO:tensorflow:Writing example 0 of 20252


INFO:tensorflow:Writing example 0 of 20252


INFO:tensorflow:*** Example ***


...

INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


Prepared train from end
Train on 20252 samples, validate on 888 samples
Epoch 1/5
20252/20252 [==============================] - 2s 114us/sample - loss: 0.0786 - val_loss: 0.0632
Epoch 2/5
20252/20252 [==============================] - 2s 95us/sample - loss: 0.0647 - val_loss: 0.0556
Epoch 3/5
20252/20252 [==============================] - 2s 94us/sample - loss: 0.0611 - val_loss: 0.0597
Epoch 4/5
20252/20252 [==============================] - 2s 95us/sample - loss: 0.0577 - val_loss: 0.0518
Epoch 5/5
20252/20252 [==============================] - 2s 100us/sample - loss: 0.0556 - val_loss: 0.0540


In [7]:
from bert_models import base_bert_model
base_bert_model = reload(base_bert_model)
base_bert_model_meta_model_with_window_all = reload(base_bert_model_meta_model_with_window_all)
study_type_labeler_window_agg = reload(study_type_labeler_window_agg)
_study_type_labeler = study_type_labeler_window_agg.StudyTypeLabeler(
      "study_type_multi",
      max_seq_length = 256, batch_size = 16,
      gpu_device_num_hub = 0, gpu_device_num = 0,
      model_folder = "../tmp/scibert_scivocab_uncased",
      label_list = list(range(len(study_labels))),
      meta_model_folder = "study_type_multi_meta_agg",
      use_one_layer=True,
      keep_prob=0.8,
      epochs_num=4,
      multilabel=True
      )
#_study_type_labeler.train_model(train, test, use_tails=True)
_study_type_labeler.train_meta_model(train, test, use_tail=True)

Started tokenizer loading
Used gpu 0
Tokenizrer loaded
0
../tmp/scibert_scivocab_uncased
INFO:tensorflow:Using config: {'_model_dir': 'study_type_multi', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000022449773898>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': 'study_type_multi', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000022449773898>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Config is done
INFO:tensorflow:Writing example 0 of 20252


INFO:tensorflow:Writing example 0 of 20252


INFO:tensorflow:*** Example ***


...

INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


Train on 20252 samples, validate on 888 samples
Epoch 1/4
20252/20252 [==============================] - 4s 214us/sample - loss: 0.0734 - val_loss: 0.0499
Epoch 2/4
20252/20252 [==============================] - 4s 201us/sample - loss: 0.0605 - val_loss: 0.0544
Epoch 3/4
20252/20252 [==============================] - 4s 195us/sample - loss: 0.0545 - val_loss: 0.0418
Epoch 4/4
20252/20252 [==============================] - 4s 184us/sample - loss: 0.0505 - val_loss: 0.0432


In [45]:
dfs_data = excel_reader.ExcelReader().read_df_from_excel("../notebooks/Studies for visualizations (1).xlsx")

Read file ../notebooks/Studies for visualizations (1).xlsx: 0.08s
Processed file ../notebooks/Studies for visualizations (1).xlsx: 0.06s


In [46]:
results = _study_type_labeler.predict_for_df(dfs_data[["title", "abstract"]])

INFO:tensorflow:Writing example 0 of 159


INFO:tensorflow:Writing example 0 of 159


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Restoring parameters from study_type_multi\model.ckpt-2000


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [75]:
dfs_data_true_labels = []
dfs_data_predicted_labels = []
study_type_dicts = {
 'Observational study': 'Qualitative',
 'Modeling study': 'Quantitative',
 'Laboratory study': 'Quantitative',
 'Review paper': 'Qualitative',
 'Field study': 'Quantitative'
}
for i in range(len(dfs_data)):
    if dfs_data["Study Design"].values[i].strip():
        
        found_labels = set()
        for idx, label in enumerate(study_labels):
            if results[1][i][idx] == 1 and idx != 3:
                found_labels.add(study_type_dicts[label])
        if len(found_labels) == 0:
            continue
        elif len(found_labels) == 1:
            dfs_data_predicted_labels.append(list(found_labels)[0])
        else:
            dfs_data_predicted_labels.append("Mixed methods")
        dfs_data_true_labels.append(dfs_data["Study Design"].values[i].strip())

In [102]:
_study_type_labeler.predict_for_df(dfs_data[["title", "abstract"]][152:153])

INFO:tensorflow:Writing example 0 of 2


INFO:tensorflow:Writing example 0 of 2


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


([array([0.5679293 , 0.05406347, 0.01991221, 0.11234513, 0.03242701],
        dtype=float32)], [[1, 0, 0, 0, 0]], [-1])

In [100]:
dfs_data["title"].values[143]

' Cost analysis of nutrition messaging intervention through community-led videos in Odisha'

In [91]:
from sklearn.feature_extraction.text import CountVectorizer
_count_vec = CountVectorizer(binary=True, analyzer=lambda x: x)
transformed_val = _count_vec.fit_transform(dfs_data["Study Methods"].apply(lambda x: [n.strip() for n in x.split("|")])).toarray()
pd_counts = pd.DataFrame(transformed_val, columns= [ "methods_ " + a for a in _count_vec.get_feature_names()])

In [94]:
pd.concat([pd.get_dummies(dfs_data["Study Design"]), pd_counts], axis=1).corr()

,,Mixed methods,Qualitative,Quantitative,methods_,methods_ Action research experiment (similar to Random Control Trial (RCT)),methods_ Bank transaction data,methods_ Data from government and DFS,methods_ Interview/focus group,methods_ Observation,...,methods_ Quiz,methods_ Random Control Trial (RCT),methods_ Self-reported survey,methods_ Service data,methods_ Similar to Random Control Trial (RCT) (Quasi-experiment),methods_ Simple field experiment (Quasi-Random Control Trial (RCT)?),methods_ Transaction data,methods_ Willingness to pay experiment,methods_ on-farm trials,methods_ pseudo-Random Control Trial (RCT)
,1.000000,-0.160422,-0.089562,-0.160422,0.808452,-0.015754,-0.015754,-0.015754,-0.206948,-0.048507,...,-0.015754,-0.087557,-0.148036,-0.053988,-0.015754,-0.015754,-0.015754,-0.015754,-0.015754,-0.015754
Mixed methods,-0.160422,1.000000,-0.366378,-0.656250,-0.087142,-0.064448,-0.064448,-0.064448,0.466274,-0.087142,...,-0.064448,-0.045264,0.332892,0.336535,-0.064448,-0.064448,-0.064448,0.098206,-0.064448,-0.064448
Qualitative,-0.089562,-0.366378,1.000000,-0.366378,-0.110782,-0.035980,-0.035980,-0.035980,0.399242,0.324141,...,-0.035980,-0.199966,-0.303161,-0.123299,-0.035980,-0.035980,-0.035980,-0.035980,-0.035980,-0.035980
Quantitative,-0.160422,-0.656250,-0.366378,1.000000,-0.142787,0.098206,0.098206,0.098206,-0.692125,-0.142787,...,0.098206,0.232879,-0.042497,-0.220851,0.098206,0.098206,0.098206,-0.064448,0.098206,0.098206
methods_,0.808452,-0.087142,-0.110782,-0.142787,1.000000,-0.019487,-0.019487,-0.019487,-0.255981,-0.060000,...,-0.019487,-0.108302,-0.183110,-0.066779,-0.019487,-0.019487,-0.019487,-0.019487,-0.019487,-0.019487
methods_ Action research experiment (similar to Random Control Trial (RCT)),-0.015754,-0.064448,-0.035980,0.098206,-0.019487,1.000000,-0.006329,-0.006329,-0.083139,-0.019487,...,-0.006329,-0.035175,-0.059471,-0.021689,-0.006329,-0.006329,-0.006329,-0.006329,-0.006329,-0.006329
methods_ Bank transaction data,-0.015754,-0.064448,-0.035980,0.098206,-0.019487,-0.006329,1.000000,-0.006329,-0.083139,-0.019487,...,-0.006329,-0.035175,-0.059471,-0.021689,-0.006329,-0.006329,-0.006329,-0.006329,-0.006329,-0.006329
methods_ Data from government and DFS,-0.015754,-0.064448,-0.035980,0.098206,-0.019487,-0.006329,-0.006329,1.000000,-0.083139,-0.019487,...,-0.006329,-0.035175,-0.059471,-0.021689,-0.006329,-0.006329,-0.006329,-0.006329,-0.006329,-0.006329
methods_ Interview/focus group,-0.206948,0.466274,0.399242,-0.692125,-0.255981,-0.083139,-0.083139,-0.083139,1.000000,0.179906,...,-0.083139,-0.291835,0.058950,0.211259,-0.083139,-0.083139,-0.083139,0.076127,-0.083139,-0.083139
methods_ Observation,-0.048507,-0.087142,0.324141,-0.142787,-0.060000,-0.019487,-0.019487,-0.019487,0.179906,1.000000,...,-0.019487,-0.034712,-0.069603,-0.066779,-0.019487,-0.019487,-0.019487,-0.019487,-0.019487,-0.019487


In [77]:
print(confusion_matrix(y_true=dfs_data_true_labels, y_pred=dfs_data_predicted_labels))
print(classification_report(y_true=dfs_data_true_labels, y_pred=dfs_data_predicted_labels))

[[ 3 42  3]
 [ 5 16  1]
 [12 33 11]]
               precision    recall  f1-score   support

Mixed methods       0.15      0.06      0.09        48
  Qualitative       0.18      0.73      0.28        22
 Quantitative       0.73      0.20      0.31        56

    micro avg       0.24      0.24      0.24       126
    macro avg       0.35      0.33      0.23       126
 weighted avg       0.41      0.24      0.22       126



In [13]:
test_plus_stefan = excel_reader.ExcelReader().read_df_from_excel("../tmp/study_type_data/final_study_type_test_data.xlsx")
result = _study_type_labeler.predict_for_df(test_plus_stefan)

Read file ../tmp/study_type_data/final_study_type_test_data.xlsx: 0.16s
Processed file ../tmp/study_type_data/final_study_type_test_data.xlsx: 0.02s
INFO:tensorflow:Writing example 0 of 250


INFO:tensorflow:Writing example 0 of 250


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Restoring parameters from study_type_multi\model.ckpt-2000


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [8]:
test_plus_stefan = excel_reader.ExcelReader().read_df_from_excel("../tmp/study_type_data/final_study_type_test_data.xlsx")
result = _study_type_labeler.predict_with_meta_model(test_plus_stefan, with_head_tail = True)

Read file ../tmp/study_type_data/final_study_type_test_data.xlsx: 0.10s
Processed file ../tmp/study_type_data/final_study_type_test_data.xlsx: 0.02s
INFO:tensorflow:Writing example 0 of 250


INFO:tensorflow:Writing example 0 of 250


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Restoring parameters from study_type_multi\model.ckpt-2000


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [5]:
from bert_models import base_bert_model_meta_model
base_bert_model_meta_model = reload(base_bert_model_meta_model)
from bert_models import base_bert_model_meta_model_with_window_all
base_bert_model_meta_model_with_window_all = reload(base_bert_model_meta_model_with_window_all)
from study_design_type import study_type_labeler
study_type_labeler = reload(study_type_labeler)
from study_design_type import study_type_labeler_window_agg
study_type_labeler_window_agg = reload(study_type_labeler_window_agg)

test_plus_stefan = excel_reader.ExcelReader().read_df_from_excel("../tmp/study_type_data/final_study_type_test_data.xlsx")
for label in StudyTypeLabels.STUDY_TYPE_LABEL_TO_NUMBER:
    if label == "No category":
        continue
    print(label)
    _study_type_labeler = study_type_labeler_window_agg.StudyTypeLabeler(
        #"/hdd2/data/maryia_pavlovets/study_type_exps_3/study_type_with_tails_%s"%label,
        "../tmp/study_type_exps_3/study_type_with_tails_%s"%label,
        max_seq_length = 256, batch_size = 16,
      gpu_device_num_hub = 0, gpu_device_num = 0,
      model_folder = "../tmp/scibert_scivocab_uncased",
      label_list = [0,1],
      meta_model_folder = "../tmp/study_type_exps_3/meta_model_%s_1layer_0.8_new_meta"%label, #"/hdd2/data/maryia_pavlovets/study_type_exps_3/meta_model_%s_1layer_0.8_new_meta"%label,
      use_one_layer=True,
      keep_prob=0.8,
      epochs_num=8
      )
    train, test = load_train_study_type(label, proportions_to_use=2)
    print(train["label"].value_counts())
    print(test["label"].value_counts())
    #_study_type_labeler.train_model(train, test, use_tails=True)
    #_study_type_labeler.train_meta_model(train, test, use_tail=True)
    r = _study_type_labeler.predict_with_meta_model(test_plus_stefan, with_head_tail = True)
    test_plus_stefan[label] = 0.0
    for i in range(len(r[0])):
        test_plus_stefan[label].values[i] = r[0][i][1]

Read file ../tmp/study_type_data/final_study_type_test_data.xlsx: 0.12s
Processed file ../tmp/study_type_data/final_study_type_test_data.xlsx: 0.01s
Observational study
Started tokenizer loading
Used gpu 0
Tokenizrer loaded
0
../tmp/scibert_scivocab_uncased
INFO:tensorflow:Using config: {'_model_dir': '../tmp/study_type_exps_3/study_type_with_tails_Observational study', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000023702D99EB8>, '_task_type': 'worker', '_task_id': 0, '_glo

INFO:tensorflow:Using config: {'_model_dir': '../tmp/study_type_exps_3/study_type_with_tails_Observational study', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000023702D99EB8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Config is done
labelled_multi.xlsx 4
Read file ../tmp/study_type_data\labelled_multi.xlsx: 0.16s
Processed file ../tmp/study_type_data\labelled_multi.xlsx: 0.04s
Read file ../tmp/study_type_data\labelled_multi.xlsx: 0.11s
Processed file ../tmp/study_type_data\labelled_multi.xlsx: 0.04s
Read file ../tmp/study_type_data\labelled_multi.xlsx: 0.13s
Processed file ../tmp/study_type_data\labelled_multi.xlsx: 0.04s
Read file ../tmp/study_type_data\labelled_multi.xlsx: 0.11s
Processed file ../tmp/study_type_data\labelled_multi.xlsx: 0.04s
train_keywords_plus_multi.xlsx 1
Read file ../tmp/study_type_data\train_keywords_plus_multi.xlsx: 1.82s
Processed file ../tmp/study_type_data\train_keywords_plus_multi.xlsx: 1.06s
valid_data.xlsx 1
Read file ../tmp/study_type_data\valid_data.xlsx: 0.11s
Processed file ../tmp/study_type_data\valid_data.xlsx: 0.07s
Read file ../tmp/study_type_data\valid_data.xlsx: 0.10s
Processed file ../tmp/study_type_data\valid_data.xlsx: 0.07s
Train  20002
0    12416
1     7

INFO:tensorflow:Writing example 0 of 250


INFO:tensorflow:*** Example ***


...

INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


AttributeError: 'str' object has no attribute 'decode'

In [3]:
def predict_proba(doc):
    return np.asarray(_study_type_labeler.predict_with_meta_model(pd.DataFrame([("title", d) for d in doc], columns=["title", "abstract"]), with_head_tail = True, recreate_model = True)[0])

In [4]:
idx = 15
text = test_plus_stefan["abstract"].values[idx]
for column in ['Observational study', 'Modeling study', 'Laboratory study', 'Review paper', 'Field study']:
    if test_plus_stefan[column].values[idx] > 0.5:
        print(column)

Observational study
Modeling study


In [5]:
from bert_models import base_bert_model_meta_model
base_bert_model_meta_model = reload(base_bert_model_meta_model)
from study_design_type import study_type_labeler
study_type_labeler = reload(study_type_labeler)
import eli5
from eli5.lime import TextExplainer

label = "Observational study"
_study_type_labeler = study_type_labeler.StudyTypeLabeler(
            "/hdd2/data/maryia_pavlovets/study_type_exps/study_type_with_tails_%s"%label,
            max_seq_length = 256, batch_size = 16,
          gpu_device_num_hub = 0, gpu_device_num = 0,
          model_folder = "../model/scibert_scivocab_uncased",
          label_list = [0,1],
          meta_model_folder = "/hdd2/data/maryia_pavlovets/study_type_exps/meta_model_%s_2layer_0.8"%label,
          use_one_layer=False,
          keep_prob=0.8,
          epochs_num=8
          )
te = TextExplainer(random_state=42)
te.fit(text, predict_proba)
te.show_prediction(target_names=[0,1])

Started tokenizer loading
Used gpu 0
Tokenizrer loaded
0
../model/scibert_scivocab_uncased
Config is done
Used for model gpu 0

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Used for model gpu 0


In [9]:
test_plus_stefan_true = excel_reader.ExcelReader().read_df_from_excel("../tmp/study_type_data/final_study_type_test_data.xlsx")
results_true = []
for i in range(len(test_plus_stefan_true)):
    results_ind = []
    for column in ['Observational study', 'Modeling study', 'Laboratory study', 'Review paper', 'Field study']:
        if test_plus_stefan_true[column].values[i] == 1:
            results_ind.append(column)
    if not results_ind:
        results_ind.append("No category")
    results_true.append(results_ind)

Read file ../tmp/study_type_data/final_study_type_test_data.xlsx: 0.04s
Processed file ../tmp/study_type_data/final_study_type_test_data.xlsx: 0.01s


In [14]:
model_results = []
for i in range(len(result[0])):
    results_ind = []
    for idx, column in enumerate(['Observational study', 'Modeling study', 'Laboratory study', 'Review paper', 'Field study']):
        if result[0][i][idx] >= 0.5:
            results_ind.append(column)
    if not results_ind:
        results_ind.append("No category")
    model_results.append(results_ind)

In [ ]:
model_results = []
for i in range(len(result)):
    results_ind = []
    for column in ['Observational study', 'Modeling study', 'Laboratory study', 'Review paper', 'Field study']:
        if test_plus_stefan[column].values[i] >= 0.5:
            results_ind.append(column)
    if not results_ind:
        results_ind.append("No category")
    model_results.append(results_ind)

In [15]:
#final model with agg meta
calculate_metrics(results_true, model_results)

Observational study
[[136   6]
 [ 19  89]]
              precision    recall  f1-score   support

           0       0.88      0.96      0.92       142
           1       0.94      0.82      0.88       108

   micro avg       0.90      0.90      0.90       250
   macro avg       0.91      0.89      0.90       250
weighted avg       0.90      0.90      0.90       250

F1 score:  0.8963361032326549
Modeling study
[[181   9]
 [  9  51]]
              precision    recall  f1-score   support

           0       0.95      0.95      0.95       190
           1       0.85      0.85      0.85        60

   micro avg       0.93      0.93      0.93       250
   macro avg       0.90      0.90      0.90       250
weighted avg       0.93      0.93      0.93       250

F1 score:  0.9013157894736841
Laboratory study
[[216   0]
 [  6  28]]
              precision    recall  f1-score   support

           0       0.97      1.00      0.99       216
           1       1.00      0.82      0.90        34

 

In [11]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import f1_score

def calculate_metrics(results_true, model_results):
    for label in StudyTypeLabels.STUDY_TYPE_LABEL_TO_NUMBER:
        print(label)
        res_y = [(1 if label in labels else 0) for labels in model_results]
        test_y = [(1 if label in labels else 0) for labels in results_true]
        print(confusion_matrix(test_y, res_y))
        print(classification_report(test_y, res_y))
        print("F1 score: ",f1_score(test_y, res_y, average="macro"))

    cnt = len(results_true)
    cnt_precision = 0
    cnt_recall = 0
    cnt_f1 = 0
    cnt_some_not_found = 0
    test_y_one = []
    res_y_one = []
    for i in range(len(results_true)):
        cnt_intersect = len(set(results_true[i]).intersection(set(model_results[i])))
        if cnt_intersect > 0:
            intersected = list(set(results_true[i]).intersection(set(model_results[i])))
            priority_labels_data = [
                (label, priority_labels[label]) for label in intersected]
            if len(priority_labels_data):
                result_label = sorted(priority_labels_data, key=lambda x: x[1])[0][0]
            else:
                result_label = 5
            test_y_one.append(result_label)
            res_y_one.append(result_label)
        else:
            priority_labels_data = [
                (label, priority_labels[label]) for label in results_true[i]]
            if len(priority_labels_data):
                result_label = sorted(priority_labels_data, key=lambda x: x[1])[0][0]
            else:
                result_label = 5
            test_y_one.append(result_label)
            priority_labels_data = [
                (label, priority_labels[label]) for label in model_results[i]]
            if len(priority_labels_data):
                result_label = sorted(priority_labels_data, key=lambda x: x[1])[0][0]
            else:
                result_label = 5
            res_y_one.append(result_label)
        cnt_correct = len(results_true[i])
        cnt_found = len(model_results[i])
        precision = 0 if cnt_found == 0 else (cnt_intersect/cnt_found)
        recall = 0 if cnt_correct == 0 else (cnt_intersect/cnt_correct)
        f1 = 0
        if (precision + recall) > 0:
            f1 = 2*precision*recall/(precision + recall)
        cnt_precision += precision
        cnt_recall += recall
        cnt_f1 += f1
    print(cnt_precision/cnt, cnt_recall/cnt, cnt_f1/cnt)
    print(confusion_matrix(test_y_one, res_y_one))
    print(classification_report(test_y_one, res_y_one))
    print("F1 score: ",f1_score(test_y_one, res_y_one, average="macro"))

In [24]:
#one vs others model
calculate_metrics(results_true, model_results)

Observational study
[[133   9]
 [ 39  69]]
              precision    recall  f1-score   support

           0       0.77      0.94      0.85       142
           1       0.88      0.64      0.74       108

   micro avg       0.81      0.81      0.81       250
   macro avg       0.83      0.79      0.79       250
weighted avg       0.82      0.81      0.80       250

F1 score:  0.7945346209163755
Modeling study
[[175  15]
 [  8  52]]
              precision    recall  f1-score   support

           0       0.96      0.92      0.94       190
           1       0.78      0.87      0.82        60

   micro avg       0.91      0.91      0.91       250
   macro avg       0.87      0.89      0.88       250
weighted avg       0.91      0.91      0.91       250

F1 score:  0.8786177197019274
Laboratory study
[[207   9]
 [  2  32]]
              precision    recall  f1-score   support

           0       0.99      0.96      0.97       216
           1       0.78      0.94      0.85        34

 

In [6]:
_study_type_labeler.generate_test_file("../notebooks/study_type_all_left_dataset_5_filtered.xlsx","new_sample_for_test.xlsx",size=80, withBoost=True,use_tail=True)

5    45482
1    18173
0     6057
3     1569
4      135
2       43
Name: label, dtype: int64
Used for model gpu 2

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Used for model gpu 2
Used for model gpu 2
Saving...
Saved to new_sample_for_test.xlsx
Saving...
Saved to ../notebooks/study_type_all_left_dataset_5_filtered.xlsx


In [25]:
from time import time
from bert_models import base_bert_model_meta_model
base_bert_model_meta_model = reload(base_bert_model_meta_model)
from bert_models import base_bert_model_meta_model_with_window_all
base_bert_model_meta_model_with_window_all = reload(base_bert_model_meta_model_with_window_all)
from study_design_type import study_type_labeler
study_type_labeler = reload(study_type_labeler)
from study_design_type import study_type_labeler_window_agg
study_type_labeler_window_agg = reload(study_type_labeler_window_agg)


for folder in ["covid_data", "dfs_data", "teams_dataset", "big_dataset_1"]:
    for filename in os.listdir(os.path.join("../tmp", folder)):
        print(folder, filename)
        new_folder = (folder+"_new") if folder == "big_dataset_1" else folder
        print(new_folder)
        if folder != new_folder and os.path.exists(os.path.join("../tmp", new_folder, filename)):
            print("Already processed")
            continue
        start = time()
        df = excel_reader.ExcelReader().read_df_from_excel(os.path.join("../tmp", folder, filename))
        df["study_type"] = ""
        for i in range(len(df)):
            df["study_type"].values[i] = []
        print(len(df))

        for label in StudyTypeLabels.STUDY_TYPE_LABEL_TO_NUMBER:
            if label == "No category":
                continue
            print(label)
            _study_type_labeler = study_type_labeler_window_agg.StudyTypeLabeler(
                "/hdd2/data/maryia_pavlovets/study_type_exps_3/study_type_with_tails_%s"%label,
                max_seq_length = 256, batch_size = 16,
              gpu_device_num_hub = 0, gpu_device_num = 0,
              model_folder = "../model/scibert_scivocab_uncased",
              label_list = [0,1],
              meta_model_folder = "/hdd2/data/maryia_pavlovets/study_type_exps_3/meta_model_%s_1layer_0.8_new_meta"%label,
              use_one_layer=True,
              keep_prob=0.8,
              epochs_num=8
              )
            
            r = _study_type_labeler.predict_with_meta_model(df[["title", "abstract"]], with_head_tail = True)
            for i in range(len(r[0])):
                if r[0][i][1] >= 0.5:
                    df["study_type"].values[i].append(label)
        
        for i in range(len(df)):
            if not len(df["study_type"].values[i]):
                df["study_type"].values[i] = ["No category"]
        print("Finished ", (time() - start))
        os.makedirs(os.path.join("../tmp", new_folder), exist_ok=True)
        excel_writer.ExcelWriter().save_df_in_excel(df, os.path.join("../tmp", new_folder, filename))

covid_data 0.xlsx
covid_data
Read file ../tmp/covid_data/0.xlsx: 0.13s
Processed file ../tmp/covid_data/0.xlsx: 0.25s
510
Observational study
Started tokenizer loading
Used gpu 0
Tokenizrer loaded
0
../model/scibert_scivocab_uncased
Config is done
Used for model gpu 0
Used for model gpu 0
Used for model gpu 0
Modeling study
Started tokenizer loading
Used gpu 0
Tokenizrer loaded
0
../model/scibert_scivocab_uncased
Config is done
Used for model gpu 0
Used for model gpu 0
Used for model gpu 0
Laboratory study
Started tokenizer loading
Used gpu 0
Tokenizrer loaded
0
../model/scibert_scivocab_uncased
Config is done
Used for model gpu 0
Used for model gpu 0
Used for model gpu 0
Review paper
Started tokenizer loading
Used gpu 0
Tokenizrer loaded
0
../model/scibert_scivocab_uncased
Config is done
Used for model gpu 0
Used for model gpu 0
Used for model gpu 0
Field study
Started tokenizer loading
Used gpu 0
Tokenizrer loaded
0
../model/scibert_scivocab_uncased
Config is done
Used for model gpu 

Used for model gpu 0
Used for model gpu 0
Used for model gpu 0
Modeling study
Started tokenizer loading
Used gpu 0
Tokenizrer loaded
0
../model/scibert_scivocab_uncased
Config is done
Used for model gpu 0
Used for model gpu 0
Used for model gpu 0
Laboratory study
Started tokenizer loading
Used gpu 0
Tokenizrer loaded
0
../model/scibert_scivocab_uncased
Config is done
Used for model gpu 0
Used for model gpu 0
Used for model gpu 0
Review paper
Started tokenizer loading
Used gpu 0
Tokenizrer loaded
0
../model/scibert_scivocab_uncased
Config is done
Used for model gpu 0
Used for model gpu 0
Used for model gpu 0
Field study
Started tokenizer loading
Used gpu 0
Tokenizrer loaded
0
../model/scibert_scivocab_uncased
Config is done
Used for model gpu 0
Used for model gpu 0
Used for model gpu 0
Finished  2112.0331161022186
Saving...
Saved to ../tmp/big_dataset_1_new/27.xlsx
big_dataset_1 45.xlsx
big_dataset_1_new
Read file ../tmp/big_dataset_1/45.xlsx: 2.48s
Processed file ../tmp/big_dataset_1/4

Used for model gpu 0
Used for model gpu 0
Laboratory study
Started tokenizer loading
Used gpu 0
Tokenizrer loaded
0
../model/scibert_scivocab_uncased
Config is done
Used for model gpu 0
Used for model gpu 0
Used for model gpu 0
Review paper
Started tokenizer loading
Used gpu 0
Tokenizrer loaded
0
../model/scibert_scivocab_uncased
Config is done
Used for model gpu 0
Used for model gpu 0
Used for model gpu 0
Field study
Started tokenizer loading
Used gpu 0
Tokenizrer loaded
0
../model/scibert_scivocab_uncased
Config is done
Used for model gpu 0
Used for model gpu 0
Used for model gpu 0
Finished  2332.4124596118927
Saving...
Saved to ../tmp/big_dataset_1_new/6.xlsx
big_dataset_1 11.xlsx
big_dataset_1_new
Read file ../tmp/big_dataset_1/11.xlsx: 2.39s
Processed file ../tmp/big_dataset_1/11.xlsx: 5.19s
10000
Observational study
Started tokenizer loading
Used gpu 0
Tokenizrer loaded
0
../model/scibert_scivocab_uncased
Config is done
Used for model gpu 0
Used for model gpu 0
Used for model gpu

Used for model gpu 0
Used for model gpu 0
Review paper
Started tokenizer loading
Used gpu 0
Tokenizrer loaded
0
../model/scibert_scivocab_uncased
Config is done
Used for model gpu 0
Used for model gpu 0
Used for model gpu 0
Field study
Started tokenizer loading
Used gpu 0
Tokenizrer loaded
0
../model/scibert_scivocab_uncased
Config is done
Used for model gpu 0
Used for model gpu 0
Used for model gpu 0
Finished  2455.3782444000244
Saving...
Saved to ../tmp/big_dataset_1_new/40.xlsx
big_dataset_1 32.xlsx
big_dataset_1_new
Read file ../tmp/big_dataset_1/32.xlsx: 2.31s
Processed file ../tmp/big_dataset_1/32.xlsx: 5.06s
10000
Observational study
Started tokenizer loading
Used gpu 0
Tokenizrer loaded
0
../model/scibert_scivocab_uncased
Config is done
Used for model gpu 0
Used for model gpu 0
Used for model gpu 0
Modeling study
Started tokenizer loading
Used gpu 0
Tokenizrer loaded
0
../model/scibert_scivocab_uncased
Config is done
Used for model gpu 0
Used for model gpu 0
Used for model gpu 

Used for model gpu 0
Used for model gpu 0
Used for model gpu 0
Finished  2571.327895641327
Saving...
Saved to ../tmp/big_dataset_1_new/9.xlsx
big_dataset_1 28.xlsx
big_dataset_1_new
Read file ../tmp/big_dataset_1/28.xlsx: 2.28s
Processed file ../tmp/big_dataset_1/28.xlsx: 5.00s
10000
Observational study
Started tokenizer loading
Used gpu 0
Tokenizrer loaded
0
../model/scibert_scivocab_uncased
Config is done
Used for model gpu 0
Used for model gpu 0
Used for model gpu 0
Modeling study
Started tokenizer loading
Used gpu 0
Tokenizrer loaded
0
../model/scibert_scivocab_uncased
Config is done
Used for model gpu 0
Used for model gpu 0
Used for model gpu 0
Laboratory study
Started tokenizer loading
Used gpu 0
Tokenizrer loaded
0
../model/scibert_scivocab_uncased
Config is done
Used for model gpu 0
Used for model gpu 0
Used for model gpu 0
Review paper
Started tokenizer loading
Used gpu 0
Tokenizrer loaded
0
../model/scibert_scivocab_uncased
Config is done
Used for model gpu 0
Used for model g

Read file ../tmp/big_dataset_1/50.xlsx: 2.15s
Processed file ../tmp/big_dataset_1/50.xlsx: 4.47s
10000
Observational study
Started tokenizer loading
Used gpu 0
Tokenizrer loaded
0
../model/scibert_scivocab_uncased
Config is done
Used for model gpu 0
Used for model gpu 0
Used for model gpu 0
Modeling study
Started tokenizer loading
Used gpu 0
Tokenizrer loaded
0
../model/scibert_scivocab_uncased
Config is done
Used for model gpu 0
Used for model gpu 0
Used for model gpu 0
Laboratory study
Started tokenizer loading
Used gpu 0
Tokenizrer loaded
0
../model/scibert_scivocab_uncased
Config is done
Used for model gpu 0
Used for model gpu 0
Used for model gpu 0
Review paper
Started tokenizer loading
Used gpu 0
Tokenizrer loaded
0
../model/scibert_scivocab_uncased
Config is done
Used for model gpu 0
Used for model gpu 0
Used for model gpu 0
Field study
Started tokenizer loading
Used gpu 0
Tokenizrer loaded
0
../model/scibert_scivocab_uncased
Config is done
Used for model gpu 0
Used for model gp

Used for model gpu 0
Used for model gpu 0
Used for model gpu 0
Modeling study
Started tokenizer loading
Used gpu 0
Tokenizrer loaded
0
../model/scibert_scivocab_uncased
Config is done
Used for model gpu 0
Used for model gpu 0
Used for model gpu 0
Laboratory study
Started tokenizer loading
Used gpu 0
Tokenizrer loaded
0
../model/scibert_scivocab_uncased
Config is done
Used for model gpu 0
Used for model gpu 0
Used for model gpu 0
Review paper
Started tokenizer loading
Used gpu 0
Tokenizrer loaded
0
../model/scibert_scivocab_uncased
Config is done
Used for model gpu 0
Used for model gpu 0
Used for model gpu 0
Field study
Started tokenizer loading
Used gpu 0
Tokenizrer loaded
0
../model/scibert_scivocab_uncased
Config is done
Used for model gpu 0
Used for model gpu 0
Used for model gpu 0
Finished  2920.113545894623
Saving...
Saved to ../tmp/big_dataset_1_new/42.xlsx
big_dataset_1 33.xlsx
big_dataset_1_new
Read file ../tmp/big_dataset_1/33.xlsx: 2.28s
Processed file ../tmp/big_dataset_1/33

Used for model gpu 0
Used for model gpu 0
Used for model gpu 0
Laboratory study
Started tokenizer loading
Used gpu 0
Tokenizrer loaded
0
../model/scibert_scivocab_uncased
Config is done
Used for model gpu 0
Used for model gpu 0
Used for model gpu 0
Review paper
Started tokenizer loading
Used gpu 0
Tokenizrer loaded
0
../model/scibert_scivocab_uncased
Config is done
Used for model gpu 0
Used for model gpu 0
Used for model gpu 0
Field study
Started tokenizer loading
Used gpu 0
Tokenizrer loaded
0
../model/scibert_scivocab_uncased
Config is done
Used for model gpu 0
Used for model gpu 0
Used for model gpu 0
Finished  3154.107394218445
Saving...
Saved to ../tmp/big_dataset_1_new/23.xlsx
big_dataset_1 44.xlsx
big_dataset_1_new
Read file ../tmp/big_dataset_1/44.xlsx: 2.32s
Processed file ../tmp/big_dataset_1/44.xlsx: 5.18s
10000
Observational study
Started tokenizer loading
Used gpu 0
Tokenizrer loaded
0
../model/scibert_scivocab_uncased
Config is done
Used for model gpu 0
Used for model gpu